 #  A Whale off the Port(folio)

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500.

In [10]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:
1. whale_returns.csv
2. algo_returns.csv
3. sp500_history.csv

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [11]:
# Reading whale returns
whale_returns_csv = Path("../Resources/whale_returns.csv")
whale_returns_df = pd.read_csv(whale_returns_csv,index_col='Date', infer_datetime_format=True, parse_dates=True)
whale_returns_df.dropna()
whale_returns_df.head()


FileNotFoundError: [Errno 2] File b'..\\Resources\\whale_returns.csv' does not exist: b'..\\Resources\\whale_returns.csv'

In [ ]:
# Count nulls
whale_returns_df.isnull().mean()*100

In [ ]:
# Drop nulls
whale_returns_df = whale_returns_df .dropna()
whale_returns_df.isnull().sum()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_returns_csv = Path("../Resources/algo_returns.csv")
algo_returns_df = pd.read_csv(algo_returns_csv,index_col='Date', infer_datetime_format=True, parse_dates=True)
algo_returns_df.dropna()
algo_returns_df.head()

In [ ]:
# Count nulls
algo_returns_df.isnull().mean()*100

In [ ]:
# Drop nulls
algo_returns_df = algo_returns_df .dropna()
algo_returns_df.isnull().sum()

## S&P 500 Returns

Read the S&P500 Historic Closing Prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_history_csv = Path("../Resources/sp500_history.csv")
sp500_history_df = pd.read_csv(sp500_history_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp500_history_df.head()

In [ ]:
# Check Data Types
sp500_history_df.dtypes


In [ ]:
# Fix Data Types
sp500_history_df['Close'] = sp500_history_df['Close'].str.replace('$', '')
sp500_history_df['Close']
sp500_history_df['Close'] = sp500_history_df['Close'].astype('float')
sp500_history_df['Close'].dtype


In [ ]:
sp500_history_df.head()

In [ ]:
# Calculate Daily Returns
sp500_daily_returns = sp500_history_df.pct_change()
sp500_daily_returns.head()

In [ ]:
# Drop nulls
sp500_daily_returns = sp500_daily_returns.dropna()
sp500_daily_returns.isnull().sum()


In [ ]:
# Rename Column
# Check Column Name 
sp500_daily_returns.columns


In [ ]:
# Set column Name 
column=["S&P 500"]
sp500_daily_returns.columns=column
sp500_daily_returns.head()

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Concatenate all DataFrames into a single DataFrame
concatenate_df = pd.concat([whale_returns_df,algo_returns_df,sp500_daily_returns], axis="columns", join="inner")
concatenate_df.sort_index(inplace=True)
concatenate_df.head()

---

# Portfolio Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance

Calculate and Plot the daily returns and cumulative returns. 

In [ ]:
# caclculate daily returns
daily_returns = concatenate_df.pct_change()


# plot daily returns 
daily_returns.plot(figsize=(20,10), title = "Daily Returns")

In [ ]:
# Calculate culuative returns 
cumulative_returns = (1 + daily_returns).cumprod() - 1

# Plot cumulative returns
cumulative_returns.plot(figsize=(20,10), title = "Cumulative Returns")

---

## Risk

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

In [ ]:
# Box plot to visually show risk
concatenate_df.plot.box(figsize=(20,10))


In [ ]:
# Daily Standard Deviations
# Calculate the standard deviation for each portfolio. Which portfolios are riskier than the S&P 500?
daily_std = concatenate_df.std()
daily_std

In [ ]:
# Determine which portfolios are riskier than the S&P 500
daily_std = daily_std.sort_values(ascending=False)
daily_std.head()

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annualized_std = daily_std * np.sqrt(252)
annualized_std.head()

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for the S&PP 500 using a 21 day window
2. Calcualte the correlation between each stock to determine which portfolios may mimick the S&P 500
2. Calculate and plot a 60 day Beta for Berkshire Hathaway Inc compared to the S&&P 500

In [ ]:
# Calculate and plot the rolling standard deviation for the S&PP 500 using a 21 day window
sp500_daily_returns.rolling(window=21).std().plot()

In [ ]:
# Correlation
correlation = concatenate_df.corr()
correlation

In [ ]:
# Calculate Beta for a single portfolio compared to the total market (S&P 500)

berkshire_covariance = concatenate_df['BERKSHIRE HATHAWAY INC'].cov(concatenate_df['S&P 500'])
berkshire_covariance

variance = concatenate_df['S&P 500'].var()
berkshire_beta = berkshire_covariance / variance
berkshire_beta


### Challenge: Exponentially Weighted Average 

An alternative way to calculate a rollwing window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the `ewm` with a 21 day half-life.

In [ ]:
# (OPTIONAL) YOUR CODE HERE

---

## Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. (After all, if you could invest in one of two portfolios, each offered the same 10% return, yet one offered lower risk, you'd take that one, right?)

Calculate and plot the annualized Sharpe ratios for all portfolios to determine which portfolio has the best performance

In [ ]:
# Annualzied Sharpe Ratios
sharpe_ratios = (daily_returns.mean() * 252) / (daily_returns.std() * np.sqrt(252))
sharpe_ratios


 plot() these sharpe ratios using a barplot.
 On the basis of this performance metric, do our algo strategies outperform both 'the market' and the whales?

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(kind="bar", title="Sharpe Ratios")

---

# Portfolio Returns

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Read the first stock
goog_historical_csv = Path("../Resources/goog_historical.csv")
goog_historical_df = pd.read_csv(goog_historical_csv,infer_datetime_format=True, parse_dates=True)
goog_historical_df.head()

In [ ]:
# Read the second stock
aapl_historical_csv = Path("../Resources/aapl_historical.csv")
appl_historical_df = pd.read_csv(aapl_historical_csv,infer_datetime_format=True, parse_dates=True)
appl_historical_df.head()

In [ ]:
# Read the third stock
cost_historical_csv = Path("../Resources/cost_historical.csv")
cost_historical_df = pd.read_csv(cost_historical_csv,infer_datetime_format=True, parse_dates=True)
cost_historical_df.head()

In [ ]:
# Concatenate all stocks into a single DataFrame
concat_df = pd.concat([goog_historical_df,appl_historical_df,cost_historical_df], axis="rows", join="inner")
concat_df.sort_index(inplace=True)
concat_df.head()

In [ ]:
# Reset the index
goog_historical_df = pd.read_csv(goog_historical_csv,infer_datetime_format=True,index_col='Trade DATE', parse_dates=True)
appl_historical_df = pd.read_csv(aapl_historical_csv,infer_datetime_format=True,index_col='Trade DATE', parse_dates=True)
cost_historical_df = pd.read_csv(cost_historical_csv,infer_datetime_format=True,index_col='Trade DATE', parse_dates=True)
concat_df = pd.concat([goog_historical_df,appl_historical_df,cost_historical_df], axis="rows", join="inner")

In [ ]:
# Pivot the Data so that the stock tickers are the columns, the dates are the index, and the 
# values are the closing prices
concat_df = pd.concat([goog_historical_df,appl_historical_df,cost_historical_df], axis="columns", join="inner")
concat_df.head()

In [ ]:
# Drop Nulls
concat_df.dropna()
concat_df.head()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Calculate weighted portfolio returns
weights = [1/3, 1/3, 1/3]
weighted_portfolio_returns = concat_df.dot(weights)
weighted_cumulative_returns = (1 + weighted_portfolio_returns).cumprod()



## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
big_concatenate_df = pd.concat([concatenate_df,concat_df], axis="rows", join="inner")

In [ ]:
# Only compare dates where the new, custom portfolio has dates
big_concatatenate_df.head()

## Re-run the performance and risk analysis with your portfolio to see how it compares to the others

In [ ]:
# Risk
big_std = big_concatenate_df.std()
big_std = big_std.sort_values(ascending=False)
big_std.head()


In [ ]:
# Rolling
big_std.rolling(window=21).std().plot()

In [ ]:
# Beta
big_berkshire_covariance = big_concatenate_df['BERKSHIRE HATHAWAY INC'].cov(big_concatenate_df['S&P 500'])
big_berkshire_covariance

big_variance = big_concatenate_df['S&P 500'].var()
big_berkshire_beta = big_berkshire_covariance / big_variance
big_berkshire_beta

In [ ]:
# Annualzied Sharpe Ratios
big_daily_returns = big_concat_df.pct_change()
big_sharpe_ratios = (big_daily_returns.mean() * 252) / (big_daily_returns.std() * np.sqrt(252))
big_sharpe_ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot
big_sharpe_ratios.plot(kind="bar", title="Sharpe Ratios")

## Include correlation analysis to determine which stocks (if any) are correlated

In [ ]:
big_correlation = big_concatenate_df.corr()
big_correlation